# Clickbait spoiling: Rerank

In [1]:
import pandas as pd
    
def load_dataset(file_name):
    import json
    ret = []
    
    with open('/kaggle/input/dataset/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            query = i['postText']
            spoiler = i['spoiler']
            paragraphs = i['targetParagraphs']
            
            if i['tags'][0] == 'multi':
                continue
                
            assert len(query) == 1
            query = query[0]

            if len(spoiler) != 1:
                spoiler = ' '.join(spoiler)
            else:
                spoiler = spoiler[0]

            ret += [{'query': query, 'uuid': i['uuid'], 'paragraphs': paragraphs, 'spoiler': spoiler}]
            
    return pd.DataFrame(ret)


test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')
# test_pilot_200 = load_dataset('200_test.jsonl')
all_datasets = pd.concat([test_dataset, train_dataset, validation_dataset])

In [2]:
len(test_dataset)

255

# Reranking utilities

In [2]:
!pip install pygaggle

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 731.8 kB/s eta 0:00:00:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 5.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 15.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 14.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
!apt install -y default-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  default-jdk-headless default-jre fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libice-dev libpthread-stubs0-dev libsm-dev
  libx11-dev libxau-dev libxcb1-dev libxdmcp-dev libxt-dev libxtst6
  libxxf86dga1 openjdk-11-jdk openjdk-11-jdk-headless openjdk-11-jre x11-utils
  x11proto-core-dev x11proto-dev xorg-sgml-doctools xtrans-dev
Suggested packages:
  libice-doc libsm-doc libx11-doc libxcb-doc libxt-doc openjdk-11-demo
  openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  default-jdk default-jdk-headless default-jre fonts-dejavu-extra
  libatk-wrapper-java libatk-wrapper-java-jni libice-dev libpthread-stubs0-dev
  libsm-dev libx11-dev libxau-dev libxcb1-dev libxdmcp-dev libxt-dev libxtst6
  libxxf86dga1 openjdk-11-jdk openjdk-11-jdk-headless openjdk-11-jre x11-utils
  x11proto-core-dev

In [4]:
!pip install transformers==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.0/884.0 kB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.3 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.0.0
    Uninstalling transformers-4.0.0:
      Successfully uninstalled transformers-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygaggle 0.0.3.1 requires tokenizers==0.9.4, but you have tokenizers 0.8.1rc2 which is incompatible.
pygaggle 0.0.3.1 requires transformers==4.0.0, but you have transformers 3.1.0 which is incompatible.
allennlp 2.10.1 requires h5py>=3.6.0, but you have h5py 3.1.0 which is incompatible.
allennlp 2.10.

In [ ]:
!pip install torch==1.4.0

In [5]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoBERT
from pygaggle.rerank.transformer import MonoT5
from tqdm import tqdm
import torch

monoT5 = MonoT5()
monoBert = MonoBERT()

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [6]:
def rerank(datapoint, reranker):
    passages = datapoint['paragraphs']
    passages = zip(range(len(passages)), passages)
    documents = [Text(i[1], {'docid': i[0]}, 0) for i in passages]
    ret = sorted(reranker.rerank(Query(datapoint['query']), documents), key=lambda i: i.score, reverse=True)

    return [{'score': i.score, 'id': i.metadata['docid'], 'text': i.text} for i in ret]


In [7]:
def run_reranking(output_dir, model, dataset, ground_truth_file_name):
    import json

    with open(output_dir + 'predictions.json', 'w+') as preds, open(output_dir + 'passage-truth.jsonl', 'w+') as passage_truth, open(output_dir + 'phrase-truth.jsonl', 'w+') as phrase_truth, open('/kaggle/input/dataset/' + ground_truth_file_name) as f:
        for i in f:
            i_parsed = json.loads(i)
            if i_parsed['tags'][0].startswith('passage'):
                passage_truth.write(i.strip() + '\n')
            elif i_parsed['tags'][0].startswith('phrase'):
                phrase_truth.write(i.strip() + '\n')
        
        result = {}
        dataset = [i for _, i in dataset.iterrows()]
        for i in tqdm(dataset):
            result[i['uuid']] = rerank(i, model)

        preds.write(json.dumps(result))

In [8]:
run_reranking('/kaggle/working/', monoT5, all_datasets, 'test.jsonl')

  0%|          | 0/3298 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_t5.py:186: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 3298/3298 [05:15<00:00, 10.44it/s]


In [9]:
run_reranking('/kaggle/working/', monoT5, test_dataset,  'test.jsonl')

100%|██████████| 255/255 [00:24<00:00, 10.62it/s]


In [10]:
test_dataset.iloc[0]

query         Study shows that most Thais spend majority of ...
uuid                       865539b0-2373-4a6b-b1ed-b24f559e6e79
paragraphs    [A new survey says that, not taking into accou...
spoiler       "Despite being awake, they are not physically ...
Name: 0, dtype: object

In [11]:
rerank(test_dataset.iloc[0], monoT5)

[{'score': -0.00605569826439023,
  'id': 0,
  'text': 'A new survey says that, not taking into account sleeping time, Thai people spend the majority of their waking hours being completely sedentary.'},
 {'score': -0.05729840323328972,
  'id': 6,
  'text': 'That’s funny. This survey by the state-owned Thai Health Promotion Foundation said that, while Thais don’t spend much of their time drinking or smoking, they are almost always physically active. 555.'},
 {'score': -1.1208022832870483,
  'id': 4,
  'text': 'We all know about the Thai disinclination for walking anywhere, but Wachira noted that so many hours without physical activity were bad for Thai people’s health.'},
 {'score': -2.171145439147949,
  'id': 2,
  'text': '"Despite being awake, they are not physically active. They spend their day sitting inside cars, at their desk or in meeting rooms. They usually lie down while watching TV or checking their smartphones," said Dr. Wachira Pengjuntr, the Health Department\'s director gen

In [13]:
run_reranking('/kaggle/working/', monoBert, all_datasets,  'test.jsonl')

100%|██████████| 3298/3298 [11:34<00:00,  4.75it/s]


In [14]:
rerank(test_dataset.iloc[0], monoBert)

[{'score': -0.0011142243165522814,
  'id': 0,
  'text': 'A new survey says that, not taking into account sleeping time, Thai people spend the majority of their waking hours being completely sedentary.'},
 {'score': -0.0020070187747478485,
  'id': 6,
  'text': 'That’s funny. This survey by the state-owned Thai Health Promotion Foundation said that, while Thais don’t spend much of their time drinking or smoking, they are almost always physically active. 555.'},
 {'score': -0.03151143714785576,
  'id': 2,
  'text': '"Despite being awake, they are not physically active. They spend their day sitting inside cars, at their desk or in meeting rooms. They usually lie down while watching TV or checking their smartphones," said Dr. Wachira Pengjuntr, the Health Department\'s director general.'},
 {'score': -0.09865924715995789,
  'id': 4,
  'text': 'We all know about the Thai disinclination for walking anywhere, but Wachira noted that so many hours without physical activity were bad for Thai peop

# Sentence Retrieval

In [20]:
def load_dataset(file_name):
    from nltk import sent_tokenize
    import json
    ret = []
    
    with open('/kaggle/input/dataset/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            query = i['postText']
            spoiler = i['spoiler']
            paragraphs = i['targetParagraphs']
            
            if i['tags'][0] == 'multi':
                continue
                
            assert len(query) == 1
            query = query[0]
            
            if len(spoiler) != 1:
                spoiler = ' '.join(spoiler)
            else:
                spoiler = spoiler[0]
            
            sentences = []
            for paragraph in paragraphs:
                for sentence in sent_tokenize(paragraph):
                    sentences += [sentence]
            
            ret += [{'query': query, 'uuid': i['uuid'], 'paragraphs': sentences, 'spoiler': spoiler}]
            
    return ret


test_sent_dataset = load_dataset('test.jsonl')
# test_sent_dataset = load_dataset('200_test.jsonl')

In [21]:
def run_sent_reranking(output_dir, model, ground_truth_file_name):

    import json
    
    with open(output_dir + 'predictions.json', 'w') as preds, open(output_dir + 'passage-truth.jsonl', 'w') as passage_truth, open(output_dir + 'phrase-truth.jsonl', 'w') as phrase_truth, open('/kaggle/input/dataset/' + ground_truth_file_name) as f:
        for i in f:
            i_parsed = json.loads(i)
            if i_parsed['tags'][0].startswith('passage'):
                passage_truth.write(i.strip() + '\n')
            elif i_parsed['tags'][0].startswith('phrase'):
                phrase_truth.write(i.strip() + '\n')
    
        result = {}
        
        for i in tqdm(test_sent_dataset):
            result[i['uuid']] = rerank(i, model)
        
        preds.write(json.dumps(result))

In [22]:
run_sent_reranking('/kaggle/working/', monoT5, 'test.jsonl')

100%|██████████| 255/255 [00:35<00:00,  7.20it/s]


# Reranking with BM25 or QL

In [ ]:
def rerank_bm25_passage(i, k1, b, rm3):
    return rerank_bm25(
        query = i['query'],
        index = 'anserini-indexes-200test/paragraphs-' + i['uuid'] + '/index',
        k1 = k1,
        b = b,
        rm3 = rm3,
    )
    
def rerank_bm25_sent(i, k1, b, rm3):
    return rerank_bm25(
        query = i['query'],
        index = 'anserini-indexes-200test/sentences-' + i['uuid'] + '/index',
        k1 = k1,
        b = b,
        rm3 = rm3,
    )

def rerank_bm25(query, index, k1, b):
    from pyserini.search import SimpleSearcher
    import json
    
    searcher = SimpleSearcher(index)
    searcher.set_bm25(k1=k1, b=b)
    hits = searcher.search(query)
    
    ret = []
    for hit in hits:
        ret += [{
            'score': hit.score,
            'id': hit.docid,
            'text': json.loads(searcher.doc(hit.docid).raw())['contents'],
        }]
        
    return ret


def rerank_bm25(query, index, k1=float(0.9), b=float(0.4), rm3=False):
    from pyserini.search import SimpleSearcher
    import json
    
    searcher = SimpleSearcher(index)
    searcher.set_bm25(k1=k1, b=b)
    
    if rm3:
         searcher.set_rm3(fb_terms=20, fb_docs=20, original_query_weight=float(0.2))
    
    hits = searcher.search(query)
    
    ret = []
    for hit in hits:
        ret += [{
            'score': hit.score,
            'id': hit.docid,
            'text': json.loads(searcher.doc(hit.docid).raw())['contents'],
        }]
        
    return ret


In [ ]:
rerank_bm25_passage(test_dataset.iloc[0], k1=0.9, b=0.4)

In [ ]:
rerank_bm25_sent(test_dataset.iloc[0], k1=0.9, b=0.4)

In [ ]:
def run_bm25_reranking(output_dir, ranking_method, ground_truth_file_name):

    import json
    
    !mkdir -p {output_dir}
    
    with open(output_dir + 'predictions.json', 'w') as preds, open(output_dir + 'passage-truth.jsonl', 'w') as passage_truth, open(output_dir + 'phrase-truth.jsonl', 'w') as phrase_truth, open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-clickbait-spoiling/' + ground_truth_file_name) as f:
        for i in f:
            i_parsed = json.loads(i)
            if i_parsed['tags'][0].startswith('passage'):
                passage_truth.write(i.strip() + '\n')
            elif i_parsed['tags'][0].startswith('phrase'):
                phrase_truth.write(i.strip() + '\n')
    
        result = {}
        
        for _, i in tqdm(test_dataset.iterrows()):
            ret = ranking_method(i)
            
            if len(ret) == 0:
                continue
            
            result[i['uuid']] = ranking_method(i)
        
        preds.write(json.dumps(result))

#for k1 in [0.8, 0.9, 1.0]:
#    for b in [0.3, 0.4, 0.5]:
#        output_dir = 'passage-retrieval-bm25/k1=' + str(k1) + '-b=' + str(b) + '/'
#        run_bm25_reranking(output_dir, lambda i: rerank_bm25_passage(i, k1, b))
#        
#        output_dir = 'sentence-retrieval-bm25/k1=' + str(k1) + '-b=' + str(b) + '/'
#        run_bm25_reranking(output_dir, lambda i: rerank_bm25_sent(i, k1, b))
        

In [ ]:
tmp_del = test_dataset
test_dataset = test_pilot_200
run_bm25_reranking('test-200/sentence-retrieval-bm25/', lambda i: rerank_bm25_sent(i, k1=0.9, b=0.4, rm3=False), '200_test.jsonl')

test_dataset = tmp_del

In [ ]:
tmp_del = test_dataset
test_dataset = test_pilot_200
run_bm25_reranking('test-200/sentence-retrieval-bm25-prf/', lambda i: rerank_bm25_sent(i, k1=0.9, b=0.4, rm3=True), '200_test.jsonl')

test_dataset = tmp_del

In [ ]:
!ls anserini-indexes/

In [ ]:
def rerank_qld_passage(i, mu, rm3):
    return rerank_qld(
        query = i['query'],
        index = 'anserini-indexes-200test/paragraphs-' + i['uuid'] + '/index',
        mu = mu,
        rm3 = rm3,
    )
    
def rerank_qld_sent(i, mu):
    return rerank_qld(
        query = i['query'],
        index = 'anserini-indexes-200test/sentences-' + i['uuid'] + '/index',
        mu = mu,
        rm3 = rm3,
    )

def rerank_qld(query, index, mu, rm3):
    from pyserini.search import SimpleSearcher
    import json
    
    searcher = SimpleSearcher(index)
    searcher.set_qld(mu = mu)
    if rm3:
        searcher.set_rm3(fb_terms=20, fb_docs=20, original_query_weight=float(0.2))
    
    hits = searcher.search(query)
    
    ret = []
    for hit in hits:
        ret += [{
            'score': hit.score,
            'id': hit.docid,
            'text': json.loads(searcher.doc(hit.docid).raw())['contents'],
        }]
        
    return ret


In [ ]:
for mu in [float(800), float(850), float(900), float(950), float(1000), float(1050), float(1100), float(1150), float(1200),]:
        output_dir = 'passage-retrieval-qld/mu=' + str(mu) + '/'
        run_bm25_reranking(output_dir, lambda i: rerank_qld_passage(i, mu))
        
        output_dir = 'sentence-retrieval-qld/mu=' + str(mu) + '/'
        run_bm25_reranking(output_dir, lambda i: rerank_qld_sent(i, mu))
     

In [ ]:
tmp_del = test_dataset
test_dataset = test_pilot_200
run_bm25_reranking('test-200/sentence-retrieval-ax/', lambda i: rerank_qld_passage(i, float(1000), True), '200_test.jsonl')

test_dataset = tmp_del

In [ ]:
tmp_del = test_dataset
test_dataset = test_pilot_200
run_bm25_reranking('test-200/sentence-retrieval-qld+rm3/', lambda i: rerank_qld_passage(i, float(1000), True), '200_test.jsonl')

test_dataset = tmp_del

# Create Evaluation Files for BM25/QLD Passage Retrieval with

```
find /mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/sentence-retrieval-bm25/ -name '*k1=*' |grep -v json|xargs -i bash -c 'export INPUT_DIR="{}" && ./eval-in-progress-maik.sh'


find /mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/passage-retrieval-bm25/ -name '*k1=*' |grep -v json|xargs -i bash -c 'export INPUT_DIR="{}" && ./eval-in-progress-maik.sh'

find /mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/sentence-retrieval-qld/ -name '*mu=*' |grep -v json|xargs -i bash -c 'export INPUT_DIR="{}" && ./eval-in-progress-maik.sh'

find /mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/passage-retrieval-qld/ -name '*mu=*'|xargs -i bash -c 'export INPUT_DIR="{}" && ./eval-in-progress-maik.sh'
```

# Create Evaluations on Pilot Experiments

```
INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-bert/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-t5/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-bm25/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-bm25-rm3/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-bm25-prf/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-bm25-ax/" ./eval-in-progress-maik.sh



INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-qld/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-rm3/" ./eval-in-progress-maik.sh


INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-prf/" ./eval-in-progress-maik.sh

INPUT_DIR="/mnt/ceph/storage/data-tmp/2021/kibi9872/ecir22-zero-shot/src/main/jupyter/clickbait-spoiling/test-200/sentence-retrieval-ax/" ./eval-in-progress-maik.sh
```